# 4 深层神经网络

## 4.2  损失函数定义

In [16]:
import tensorflow as tf

v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

sess = tf.InteractiveSession()
# clip_by_value 可以将一个张量的数值限制在一个范围内（2.5， 4.5），这样可以避免如log0的错误
print tf.clip_by_value(v, 2.5, 4.5).eval()

v = tf.constant([1.0, 2.0, 3.0])
print tf.log(v).eval()

v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[5.0, 6.0], [7.0, 8.0]])

print (v1 * v2).eval() # 元素直接乘
print tf.matmul(v1, v2).eval() # 矩阵乘法

v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print tf.reduce_mean(v).eval() # 整个矩阵平均

# softmax 回归之后的交叉熵，多分类时
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(y, y_)
# 二分类时的交叉熵，得到优化加速
# cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, y_)

# 回归问题的均方误差
# mse = tf.reduce_mean(tf.square(y_, y))

[[ 2.5  2.5  3. ]
 [ 4.   4.5  4.5]]
[ 0.          0.69314718  1.09861231]
[[  5.  12.]
 [ 21.  32.]]
[[ 19.  22.]
 [ 43.  50.]]
3.5


In [15]:
# 自定义损失函数
# loss = tf.reduce_sum(tf.select(tf.greater(v1, v2), (v1 - v2) * a, (v2 - v1) * b))

v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])

sess = tf.InteractiveSession()
# greater比较大小并返回比较结果，维度不一致时会进行广播操作。
print tf.greater(v1, v2).eval()

# select类似于双目操作符
print tf.select(tf.greater(v1, v2), v1, v2).eval()

sess.close()

AttributeError: 'module' object has no attribute 'select'

In [17]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.select(tf.greater(y, y_), (y - y_) * loss_more, (y_ - y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
# 0.05为加入的不可预测的噪音
Y = [[x1 + x2 + rdm.rand()/10.0 - 0.05] for (x1, x2) in X]

with tf.Session as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict={x: X[start, end], y_: Y[start, end]})
        print sess.run(w1)

AttributeError: 'module' object has no attribute 'select'